In [1]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timezone, timedelta
import pytz
import time
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

def get_samples(num_samples, SAVE_PATH=''):
    
    cdata = np.empty((3,num_samples), dtype=np.complex128)
    # ADC_A is channel[3], ADC_C is channel[1], ADC_D is channel[0]
    for i, j in enumerate([3,1,0]):
        cdata[i,:] = np.array(base.radio.receiver.channel[j].transfer(num_samples))
    
    if SAVE_PATH:
        np.save('data/' + SAVE_PATH, cdata)
    
    return cdata

In [2]:
from pynq.overlays.base import BaseOverlay
import xrfdc
import xrfclk
base = BaseOverlay('base.bit')
base.init_rf_clks()

# DAC_A - Checked, working
base.radio.transmitter.channel[1].dac_block.MixerSettings['Freq'] = 1200 # MHz
base.radio.transmitter.channel[1].control.gain = 0.5
base.radio.transmitter.channel[1].control.enable = True

# DAC_B
base.radio.transmitter.channel[0].dac_block.MixerSettings['Freq'] = 1200 # MHz
base.radio.transmitter.channel[0].control.gain = 0.5
base.radio.transmitter.channel[0].control.enable = False

In [17]:
%%time
num_samples = 32768
iterations = 1

# 1x 100 pass completed, no issue
# 1x 1000 pass completed, no issue
# 2x 200 pass completed, no issue
# 1x 400 pass completed, no issue
for i in range(iterations):
    try:
        cdata = get_samples(num_samples)
    except:
        print(f'Iteration: {i}')
        raise

CPU times: user 462 ms, sys: 12.2 ms, total: 474 ms
Wall time: 860 ms


In [ ]:
%%time
num_samples = 32768
iterations = 1000

# 1x 1 pass completed, no issue
# 1x 10 pass completed, no issue
# 1x 100 pass completed, no issue
# 1x 1000 pass 
iter_array = np.empty((iterations,3,num_samples), dtype=np.complex128)
for i in range(iterations):
    try:
        cdata = get_samples(num_samples)
        iter_array[i] = np.copy(cdata)
    except:
        print(f'Iteration: {i}')
        raise